In [1]:
%load_ext autoreload
%autoreload 2

import qubx
%qubxd

from pathlib import Path
from qubx import logger, QubxLogConfig
from qubx.utils.runner.runner import run_strategy, StrategyConfig, AccountConfigurationManager, ExchangeConfig, LoggingConfig
from qubx.core.interfaces import IStrategy, IStrategyContext, BaseErrorEvent

QubxLogConfig.set_log_level("INFO")


⠀⠀⡰⡖⠒⠒⢒⢦⠀⠀   
⠀⢠⠃⠈⢆⣀⣎⣀⣱⡀  QUBX | Quantitative Backtesting Environment 
⠀⢳⠒⠒⡞⠚⡄⠀⡰⠁         (c) 2025, ver. 0.6.5
⠀⠀⠱⣜⣀⣀⣈⣦⠃⠀⠀⠀ 
        


In [ ]:
class TestStrategy(IStrategy):
    def on_error(self, ctx: IStrategyContext, error: BaseErrorEvent) -> None:
        logger.error(f"Error: {error}")

    def on_stop(self, ctx: IStrategyContext):
        logger.info("Stopping test strategy")


ctx = run_strategy(
    config=StrategyConfig(
        name="TestStrategy",
        strategy=TestStrategy,
        exchanges={
            "BINANCE.PM": ExchangeConfig(
                connector="ccxt",
                universe=["ETHUSDT"],
                params={"enable_price_match": True, "price_match_ticks": 20},
                # params={"enable_price_match": False},
            )
        },
        logging=LoggingConfig(
            logger="InMemoryLogsWriter",
            position_interval="10s",
            portfolio_interval="1m",
            heartbeat_interval="10m",
        )
    ),
    account_manager=AccountConfigurationManager(
        Path("~/releases/R_MM_olereon/accounts.toml").expanduser()
    ),
    paper=False,
)

In [3]:
instr = ctx.instruments[0]
instr

BINANCE.UM:SWAP:ETHUSDT

In [11]:
q = ctx.quote(instr)
assert q is not None
print(q)

ctx.trade_async(instr, amount=-0.02, price=q.bid, time_in_force="gtx")
# ctx.trade_async(instr, amount=0.02, price=q.bid + instr.tick_size, time_in_force="gtx")
# ctx.trade_async(instr, amount=-0.02, price=q.ask, time_in_force="gtx")
# ctx.trade(instr, amount=0.002, price=q.bid + 500, time_in_force="gtx")

[2025-03-21T15:09:45.692000000]	1950.73000 (322.5) | 1950.74000 (121.5)


In [12]:
ctx.get_orders()

{}

In [13]:
ctx.stop()